# 特定の人物が写っているかどうか判定する

In [1]:
from dotenv import load_dotenv
import glob
import os
import sys
import time
import uuid
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType

# .envファイルからキーを取得する
load_dotenv('/.env') # 皆さんのPCでの.envファイルの保存場所に合わせて変更しましょう
cog_key = os.getenv('COG_SERVICES_KEY')
cog_endpoint = os.getenv('COG_SERVICES_ENDPOINT')

face_client = FaceClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# ID用にランダムな文字列を生成する
PERSON_GROUP_ID = str(uuid.uuid4())

# 特定の人物を識別するためのPersonGroupを作成する
face_client.person_group.create(person_group_id=PERSON_GROUP_ID, name='Geoup1')

# 女性、男性、子どもを登録するための変数を設定する
# 以下のメソッドの第二引数は特定したい人物によって'Woman', 'Man', 'Child'の中から適切なものを設定する
woman = face_client.person_group_person.create(PERSON_GROUP_ID, 'Woman')
man = face_client.person_group_person.create(PERSON_GROUP_ID, 'Man')
child = face_client.person_group_person.create(PERSON_GROUP_ID, 'Child')

jpg_path = r'../data/5-4/person_detect/*.jpg' # 皆さんのPCの写真保管場所に合わせて変更してください

# ファイル名が'w'から始まるファイルをlistに格納
woman_images = [file for file in glob.glob(jpg_path) if file.split('/')[-1].startswith('w')]
# ファイル名が'm'から始まるファイルをlistに格納
man_images = [file for file in glob.glob(jpg_path) if file.split('/')[-1].startswith('m')]
# ファイル名が'ch'から始まるファイルをlistに格納
child_images = [file for file in glob.glob(jpg_path) if file.split('/')[-1].startswith('ch')]

# 顔写真を人物に割り当てる
for image in woman_images:
    w = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, woman.person_id, w)

for image in man_images:
    m = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, man.person_id, m)

for image in child_images:
    ch = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, child.person_id, ch)

# PersonGroupをトレーニングする
print()
print('Person group をトレーニング中...')
face_client.person_group.train(PERSON_GROUP_ID)

# 5秒毎にTraining statusを取得し、「成功」か「失敗」なら処理を終了する
while (True):
    training_status = face_client.person_group.get_training_status(PERSON_GROUP_ID)
    print('Training status: {}.'.format(training_status.status))
    print()
    if (training_status.status is TrainingStatusType.succeeded):
        break
    elif (training_status.status is TrainingStatusType.failed):
        face_client.person_group.delete(person_group_id=PERSON_GROUP_ID)
        sys.exit('Person group のトレーニングが失敗しました')
    time.sleep(5)


Person group をトレーニング中...
Training status: running.

Training status: succeeded.



In [2]:
# テスト画像を取得する
test_jpg_path = r'../data/5-4/person_detect/test-image-person-group.jpg' # 皆さんのPCの写真保管場所に合わせて変更してください
test_image_array = glob.glob(test_jpg_path)
image = open(test_image_array[0], 'r+b')

print('フリーアカウントの制限回避のため、60秒待機中...')
time.sleep (60)

# 顔を識別する
face_ids = []
# 「detection model 3」を使用して識別する
faces = face_client.face.detect_with_stream(image, detection_model='detection_03')
for face in faces:
    face_ids.append(face.face_id)

# 識別された顔IDを変数に格納
results = face_client.face.identify(face_ids, PERSON_GROUP_ID)

if results:
    if results[0].candidates:
        print('写真{}にPERSON GROUP ID: {}の人が写っています'.format(os.path.basename(image.name), PERSON_GROUP_ID))
    else:
        print('写真{}にPERSON GROUP ID: {}の人は写っていません'.format(os.path.basename(image.name), PERSON_GROUP_ID))
else:
    print('写真{}に人を確認できませんでした'.format(os.path.basename(image.name)))

フリーアカウントの制限回避のため、60秒待機中...
写真test-image-person-group.jpgにPERSON GROUP ID: 0465b92a-b32d-4e24-a53f-b08c36cd8ca7の人が写っています
